In [168]:
# Google Colaboratory
# 環境をアップデートする
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [2]:
!pip install pandas -U
!pip install numpy -U
!pip install pandas_profiling -U
!pip install xgboost -U

    100% |████████████████████████████████| 8.9MB 3.8MB/s 
  Found existing installation: pandas 0.22.0
    Uninstalling pandas-0.22.0:
      Successfully uninstalled pandas-0.22.0
    100% |████████████████████████████████| 13.9MB 2.6MB/s 
tensorflow 1.10.1 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.15.1 which is incompatible.
  Found existing installation: numpy 1.14.5
    Uninstalling numpy-1.14.5:
      Successfully uninstalled numpy-1.14.5
    100% |████████████████████████████████| 15.8MB 2.4MB/s 
  Found existing installation: xgboost 0.7.post4
    Uninstalling xgboost-0.7.post4:
      Successfully uninstalled xgboost-0.7.post4


In [3]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision

# GPUの確認
import torch
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [0]:
# ユーザー認証
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

In [0]:
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.cross_validation import *
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import io
# import pandas_profiling as pdp
import re

In [52]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test (1).csv


In [0]:
# Load the data
df_train = pd.read_csv('train.csv', header=0)
df_test = pd.read_csv('test.csv', header=0)
df_both = df_train.append(df_test)

In [152]:
print(df_both.columns)
print(df_both.describe())

Index(['Age', 'Cabin', 'Embarked', 'Fare', 'Name', 'Parch', 'PassengerId',
       'Pclass', 'Sex', 'SibSp', 'Survived', 'Ticket'],
      dtype='object')
               Age         Fare        Parch  PassengerId       Pclass  \
count  1046.000000  1308.000000  1309.000000  1309.000000  1309.000000   
mean     29.881138    33.295479     0.385027   655.000000     2.294882   
std      14.413493    51.758668     0.865560   378.020061     0.837836   
min       0.170000     0.000000     0.000000     1.000000     1.000000   
25%      21.000000     7.895800     0.000000   328.000000     2.000000   
50%      28.000000    14.454200     0.000000   655.000000     3.000000   
75%      39.000000    31.275000     0.000000   982.000000     3.000000   
max      80.000000   512.329200     9.000000  1309.000000     3.000000   

             SibSp    Survived  
count  1309.000000  891.000000  
mean      0.498854    0.383838  
std       1.041658    0.486592  
min       0.000000    0.000000  
25%       0.000

In [0]:
# 変数を作成する
# Cabinにいたか
df_both['in_cabin'] = 1
df_both.loc[df_both['Cabin'].isnull(), 'in_cabin'] = 0


# Cabinを複数人でシェアしていたか
df_c = df_both[['Cabin']].copy()
df_c['cabin_shared'] = 1
df_c = df_c.groupby('Cabin').sum().reset_index()

df_both = pd.merge(df_both, df_c, how='left', on=['Cabin'])
df_both['cabin_shared'].fillna(0, inplace=True)
df_both['cabin_shared'] = df_both['cabin_shared'].apply(lambda x : 2 if x > 1 else x)


# 家族数
df_both['familySize'] = df_both['SibSp'] + df_both['Parch'] + 1
df_both['is_alone'] = df_both['familySize'].apply(lambda x: 1 if x == 1 else 0)

# 敬称を抽出
df_both['Salutation'] = 'others'
df_both.loc[df_both['Name'].apply(lambda x: ', Miss.' in x), 'Salutation'] = 'Miss'
df_both.loc[df_both['Name'].apply(lambda x: ', Mlle.' in x), 'Salutation'] = 'Miss'
df_both.loc[df_both['Name'].apply(lambda x: ', Ms.' in x), 'Salutation'] = 'Miss'
df_both.loc[df_both['Name'].apply(lambda x: ', Mr.' in x), 'Salutation'] = 'Mr' 
df_both.loc[df_both['Name'].apply(lambda x: ', Mrs.' in x), 'Salutation'] = 'Mrs'
df_both.loc[df_both['Name'].apply(lambda x: ', Mme' in x), 'Salutation'] = 'Mrs'
df_both.loc[df_both['Name'].apply(lambda x: ', Sir' in x), 'Salutation'] = 'Sir'
df_both.loc[df_both['Name'].apply(lambda x: ', Master' in x), 'Salutation'] = 'Master'
df_both.loc[df_both['Name'].apply(lambda x: ', Rev.' in x), 'Salutation'] = 'Mr'
df_both.loc[df_both['Name'].apply(lambda x: ', Don.' in x), 'Salutation'] = 'Mr'
# df_both.loc[df_both['Name'].apply(lambda x: ', Dr.' in x), 'Salutation'] = 'Dr'


# 敬称別平均年齢を欠測値にあてる
df_g = df_both[['Salutation', 'Age']].groupby(['Salutation']).mean().reset_index()
df_g.rename(columns={'Age': 'Ave_Age'}, inplace=True)

df_both = pd.merge(df_both, df_g, how='left', on=['Salutation'])
df_both['Age'].fillna(df_both['Ave_Age'], inplace=True)

# Golden Ticket
df_both['golden_ticket'] = df_both['Ticket'].apply(lambda x: 1 
                                                   if (len(x) == 4) 
                                                   or (len(x) == 5 and x[0] in ('1', '2'))
                                                   or (len(x) == 6 and x[0] in ('3'))
                                                   or (len(x) == 7 and x.startswith('PP'))
                                                   or (len(x) == 7 and x.startswith('C '))
                                                   or (len(x) == 9 and x.startswith('C.'))
                                                   or (len(x) == 10 and x.startswith('C.'))
                                                   or x.startswith('PC ')
                                                   or x.startswith('STON/')
                                                   else 0)

# Embarked
df_both['Embarked_n'] = 0
df_both.loc[df_both['Embarked'] == 'Q', 'Embarked_n'] = 1
df_both.loc[df_both['Embarked'] == 'C', 'Embarked_n'] = 2

# 家族属性
df_both['is_family_boy'] = df_both.apply(lambda row: 1 if row['Salutation'] == 'Master' and row['familySize'] in (2, 3, 4) else 0, axis=1)
df_both['is_large_family'] = df_both.apply(lambda row: 1 if row['familySize'] > 4 else 0, axis=1)


In [0]:
# チケット価格
df_t = df_both[['Pclass', 'familySize', 'Fare']].groupby(['Pclass', 'familySize'], as_index=False).median().rename(columns={'Fare': 'Fare_Ave'})
df_both = pd.merge(df_both, df_t, how='left', on=['Pclass', 'familySize'])
df_both['high_fare'] = df_both.apply(lambda row: 1 if row['Fare'] > row['Fare_Ave'] else 0, axis=1)

In [155]:
df_both.columns

Index(['Age', 'Cabin', 'Embarked', 'Fare', 'Name', 'Parch', 'PassengerId',
       'Pclass', 'Sex', 'SibSp', 'Survived', 'Ticket', 'in_cabin',
       'cabin_shared', 'familySize', 'is_alone', 'Salutation', 'Ave_Age',
       'golden_ticket', 'Embarked_n', 'is_family_boy', 'is_large_family',
       'Fare_Ave', 'high_fare'],
      dtype='object')

In [0]:
nonnumeric_columns = ['Salutation', 'Sex']
feature_columns_to_use = ['Pclass', 'golden_ticket', 'is_family_boy', 'cabin_shared', 'high_fare', 'is_large_family', 'Embarked_n'] + nonnumeric_columns

In [157]:
cut_vals = [[c] for c in feature_columns_to_use]

for c in cut_vals:
  print(df_both[c + ['Survived']].groupby(c).agg({'Survived': ['mean', 'count']}))

        Survived      
            mean count
Pclass                
1       0.629630   216
2       0.472826   184
3       0.242363   491
               Survived      
                   mean count
golden_ticket                
0              0.375385   325
1              0.388693   566
               Survived      
                   mean count
is_family_boy                
0              0.368239   869
1              1.000000    22
              Survived      
                  mean count
cabin_shared                
0.0           0.299854   687
1.0           0.520000    75
2.0           0.751938   129
           Survived      
               mean count
high_fare                
0          0.390438   502
1          0.375321   389
                 Survived      
                     mean count
is_large_family                
0                0.400483   829
1                0.161290    62
            Survived      
                mean count
Embarked_n                
0           0.339

In [158]:
df_both[feature_columns_to_use].head()

,Pclass,golden_ticket,is_family_boy,cabin_shared,high_fare,is_large_family,Embarked_n,Salutation,Sex
0,3,0,0,0.0,0,0,0,Mr,male
1,1,1,0,2.0,0,0,2,Mrs,female
2,3,1,0,0.0,1,0,0,Miss,female
3,1,0,0,2.0,0,0,0,Mrs,female
4,3,1,0,0.0,1,0,0,Mr,male


In [0]:
le = LabelEncoder()
for feature in nonnumeric_columns:
    df_both[feature] = le.fit_transform(df_both[feature])

In [160]:
df_both.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,...,familySize,is_alone,Salutation,Ave_Age,golden_ticket,Embarked_n,is_family_boy,is_large_family,Fare_Ave,high_fare
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,1,...,2,0,2,32.387288,0,0,0,0,14.4542,0
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,...,2,0,3,36.918129,1,2,0,0,71.2833,0
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,0,0,...,1,1,1,21.824366,1,0,0,0,7.8542,1
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,0,1,...,2,0,3,36.918129,0,0,0,0,71.2833,0
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,1,0,...,1,1,2,32.387288,1,0,0,0,7.8542,1


In [0]:
# Prepare the inputs for the model
train_X = pd.merge(df_train[['PassengerId']], df_both[['PassengerId'] + feature_columns_to_use], how='left', on='PassengerId').drop(['PassengerId'], axis=1).as_matrix()
test_X = pd.merge(df_test[['PassengerId']], df_both[['PassengerId'] + feature_columns_to_use], how='left', on='PassengerId').drop(['PassengerId'], axis=1).as_matrix()
train_y = df_train['Survived']

In [162]:
train_X

array([[3., 0., 0., ..., 0., 2., 1.],
       [1., 1., 0., ..., 2., 3., 0.],
       [3., 1., 0., ..., 0., 1., 0.],
       ...,
       [3., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 2., 2., 1.],
       [3., 1., 0., ..., 1., 2., 1.]])

In [163]:
# scikit-learn を用いた交差検証（Cross-validation）とハイパーパラメータのチューニング（grid search）
# https://qiita.com/tomov3/items/039d4271ed30490edf7b

xgb_model = xgb.XGBClassifier()
xgb_params = {
    'objective': ['binary:logistic'],
    'learning_rate': [0.1, 0.2],     # so called `eta` value
    'gamma': [1],
    'max_depth': [4, 5],
    'min_child_weight': [10, 15],
    'silent': [1],
    'subsample': [0.5, 0.6],
    'colsample_bytree': [0.7, 0.8],
    'n_estimators': [500, 700],     # number of trees, change it to 1000 for better results
    'seed': [1337]
}

rf_model = RandomForestClassifier()
rf_params = {
    'n_estimators'      : [700],
    'max_features'      : ['auto'],
    'criterion'         : ['entropy', 'gini'],
    'random_state'      : [0],
    'n_jobs'            : [5],
    'min_samples_split' : [15, 18],
    'max_depth'         : [15, 18],
}

model = rf_model
params = rf_params

clf = GridSearchCV(model, params, n_jobs=5, 
                   cv=StratifiedKFold(train_y, n_folds=5, shuffle=True), 
                   # cv=5,
                   scoring='roc_auc',
                   verbose=2, refit=True)

clf.fit(train_X, train_y)

best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('Raw AUC score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] criterion=entropy, max_depth=15, max_features=auto, min_samples_split=15, n_estimators=700, n_jobs=5, random_state=0 
[CV] criterion=entropy, max_depth=15, max_features=auto, min_samples_split=15, n_estimators=700, n_jobs=5, random_state=0 
[CV] criterion=entropy, max_depth=15, max_features=auto, min_samples_split=15, n_estimators=700, n_jobs=5, random_state=0 
[CV] criterion=entropy, max_depth=15, max_features=auto, min_samples_split=15, n_estimators=700, n_jobs=5, random_state=0 
[CV] criterion=entropy, max_depth=15, max_features=auto, min_samples_split=15, n_estimators=700, n_jobs=5, random_state=0 
[CV]  criterion=entropy, max_depth=15, max_features=auto, min_samples_split=15, n_estimators=700, n_jobs=5, random_state=0 -   3.2s
[CV] criterion=entropy, max_depth=15, max_features=auto, min_samples_split=18, n_estimators=700, n_jobs=5, random_state=0 
[CV]  criterion=entropy, max_depth=15, max_features=auto, min_samples_

[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:   37.1s


[CV] criterion=gini, max_depth=18, max_features=auto, min_samples_split=18, n_estimators=700, n_jobs=5, random_state=0 
[CV]  criterion=gini, max_depth=18, max_features=auto, min_samples_split=15, n_estimators=700, n_jobs=5, random_state=0 -   6.1s
[CV] criterion=gini, max_depth=18, max_features=auto, min_samples_split=18, n_estimators=700, n_jobs=5, random_state=0 
[CV]  criterion=gini, max_depth=18, max_features=auto, min_samples_split=15, n_estimators=700, n_jobs=5, random_state=0 -   6.2s
[CV] criterion=gini, max_depth=18, max_features=auto, min_samples_split=18, n_estimators=700, n_jobs=5, random_state=0 
[CV]  criterion=gini, max_depth=18, max_features=auto, min_samples_split=15, n_estimators=700, n_jobs=5, random_state=0 -   5.3s
[CV] criterion=gini, max_depth=18, max_features=auto, min_samples_split=18, n_estimators=700, n_jobs=5, random_state=0 
[CV]  criterion=gini, max_depth=18, max_features=auto, min_samples_split=18, n_estimators=700, n_jobs=5, random_state=0 -   5.5s
[CV]

[Parallel(n_jobs=5)]: Done  40 out of  40 | elapsed:   45.4s finished


Raw AUC score: 0.8814898905290609
criterion: 'entropy'
max_depth: 15
max_features: 'auto'
min_samples_split: 15
n_estimators: 700
n_jobs: 5
random_state: 0


In [164]:
thres = [0.48, 0.49, 0.5, 0.51, 0.52]

train_result = pd.merge(df_train[['PassengerId']], df_both, how='left', on='PassengerId').drop(['PassengerId'], axis=1)
train_result['score'] = [p[1] for p in clf.predict_proba(train_X)]

best_thres = thres[0]
best_score = 0
for t in thres:
  train_result['pred'] = train_result['score'].apply(lambda x: int(x + t))

  train_result['match'] = train_result.apply(lambda row: 1 if row['pred'] == row['Survived'] else 0, axis=1)
  score = train_result['match'].mean()
  print(t, score)
  
  if score > best_score:
    best_thres = t
    best_score = score
    print('best thres is updated to {}'.format(t))

train_result['pred'] = train_result['score'].apply(lambda x: int(x + best_thres))

0.48 0.8619528619528619
best thres is updated to 0.48
0.49 0.8630751964085297
best thres is updated to 0.49
0.5 0.8630751964085297
0.51 0.8630751964085297
0.52 0.8630751964085297


In [165]:
thres = 0.5
thres = best_thres
test_result = df_test[['PassengerId']]
test_result['Survived'] = [int(p[1] + thres) for p in clf.predict_proba(test_X)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [166]:
for c in feature_columns_to_use:
  print(train_result[[c, 'Survived', 'match']].groupby([c, 'Survived']).agg({'match': ['mean', 'sum', 'count']}))

                    match           
                     mean  sum count
Pclass Survived                     
1      0.0       0.900000   72    80
       1.0       0.830882  113   136
2      0.0       0.938144   91    97
       1.0       0.908046   79    87
3      0.0       0.930108  346   372
       1.0       0.571429   68   119
                           match           
                            mean  sum count
golden_ticket Survived                     
0             0.0       0.960591  195   203
              1.0       0.795082   97   122
1             0.0       0.907514  314   346
              1.0       0.740909  163   220
                          match           
                           mean  sum count
is_family_boy Survived                    
0             0.0       0.92714  509   549
              1.0       0.74375  238   320
1             1.0       1.00000   22    22
                          match           
                           mean  sum count
cabin_shared Su

In [0]:
test_result.to_csv('submission.csv', index=False)
files.download('submission.csv')

In [0]:
train_result.to_csv('detail.csv', index=False)
files.download('detail.csv')